## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Привет! Поздравляю тебя с первым твоим проектом и спасибо, что вовремя сдал задание:) Ты проделал большую работу. Далее в файле мои комментарии ты сможешь найти в ячейках, аналогичных данной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта. 

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Чтение файла:

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv') 

Проверка уникальных значений некоторых столбцов:

In [2]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение количества детей -1 и 20 кажется странным

In [3]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Единственное значение, которое не допустимо это 0 значение, надем их количество в таблице, заметим, что количество строк с нулевым возрастом меньше 0,5% от общего числа строк 

In [4]:
len(df.query('dob_years == 0'))

101

In [5]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Заметим, что данные записаны в разных регистрах, при поиске дубликатов этот факт нужно учесть

In [6]:
df['family_status'].unique()
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [7]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [8]:
df.query('gender == "XNA"') 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Присутствует значение XNA , но оно всего одно на всю таблицу

In [9]:
df['income_type'].unique()
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

В данной талице имеются некорректные данные, которые мы должны будем в дальнейшем обработать

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>
Хорошее начало, так держать!)


### Шаг 2. Предобработка данных

### Обработка пропусков

Для начала разберемся с  теми некорректными данными

In [10]:
df = df.query(' 20 > children > -1') 

In [11]:
df = df.query('gender != "XNA"') 

Найдем строки, в которых стаж работы в днях больше нежели количество прожитых дней за минусом 18 лет в днях и заменим их на медиальное значение стажа работы взависимости от возраста

Заменим отрицательные значения days_employed на противоположные по модулю:

In [12]:
df.loc[df['days_employed']<0, ['days_employed']] *= -1 

Создадим таблицу стажа от возраста для корректных значений

In [13]:
df_1 = df.query('days_employed < 365*(dob_years-18)') 
df_2 = df_1.groupby('dob_years')['days_employed'].median()

Функция замены некорректных значений стажа на медиальные значения стажа по возрасту

In [14]:
def change(row): 
    if row['days_employed'] > row['dob_years']*365-6750:
        row['days_employed'] = df_2[row['dob_years']]
        return row
    else:
        return row

In [15]:
df = df[df['dob_years']!=0].apply(change, axis=1)

Проверка

In [16]:
len(df.query('days_employed > 365*(dob_years-18)')) 

0

Поиск пропусков :

In [17]:
print(df.isnull().sum()) 

children               0
days_employed       2152
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2152
purpose                0
dtype: int64


Пропущенные значения столбца total_income заменим медиальными значниями дохода для каждого типа занятости

In [18]:
types = df['income_type'].unique().tolist()
print(types)

['сотрудник', 'пенсионер', 'компаньон', 'госслужащий', 'безработный', 'предприниматель', 'студент', 'в декрете']


In [19]:
df_grouped = df.groupby('income_type')['total_income'].median()

функция замены пропусков медиальными значениями

In [20]:
for element in types: 
    df.loc[df['income_type'] == element, 'total_income'] = df.loc[df['income_type'] == element, 'total_income'].fillna(df_grouped[element])

In [21]:
print(df.isnull().sum())

children               0
days_employed       2152
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


Пропущенные значения столбца days_employed заменим медиальными значениями стажа для каждого возраста

In [22]:
types = df['dob_years'].unique().tolist()
print(types)

[42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24, 21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61, 64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66, 69, 19, 72, 70, 74, 75]


In [23]:
df_grouped = df.groupby('dob_years')['days_employed'].median()

функция замены пропусков медиальными значениями

In [24]:
for element in types: 
    df.loc[df['dob_years'] == element, 'days_employed'] = df.loc[df['dob_years'] == element, 'days_employed'].fillna(df_grouped[element])

Проверка пропущенных значений

In [25]:
print(df.isnull().sum()) 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Вывод

Пропуски имеются только в 2 столбцах: days_employed, total_income. Эти два столбца в каком-то смысле связаны между собой т.е. это данные с места фактической работы,  возможно эти люди официально не трудоустроены и у них нет документов подтверждающих их стаж и ежемесячную заработную плату, возможно еще по каким-то причинам не предоставили эти данные.  Их данные мы вряд ли узнаем, и не сможем заполнить пропуски вручную. Да и самих NaN слишком много для ручной замены.Тем не менее, избавиться от значений NaN можно, если заменить их единым значением. Например, заменив медиальными значениями дохода и стажа по типу занятости и возрасту соответственно.

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>
Всё верно, молодец


### Замена типа данных

Смотрим типы данных столбцов

In [26]:
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21524
Data columns (total 12 columns):
children            21301 non-null int64
days_employed       21301 non-null float64
dob_years           21301 non-null int64
education           21301 non-null object
education_id        21301 non-null int64
family_status       21301 non-null object
family_status_id    21301 non-null int64
gender              21301 non-null object
income_type         21301 non-null object
debt                21301 non-null int64
total_income        21301 non-null float64
purpose             21301 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB
None


 Замена типа данных float на int

In [27]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

In [28]:
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21301 entries, 0 to 21524
Data columns (total 12 columns):
children            21301 non-null int64
days_employed       21301 non-null int64
dob_years           21301 non-null int64
education           21301 non-null object
education_id        21301 non-null int64
family_status       21301 non-null object
family_status_id    21301 non-null int64
gender              21301 non-null object
income_type         21301 non-null object
debt                21301 non-null int64
total_income        21301 non-null int64
purpose             21301 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB
None


### Вывод

Так как особенность метода to_numeric() в том, что при переводе все числа будут иметь тип данных float. А по заданию нам нужно изменить вещественный тип данных на целочисленный, то этот метод нам не подходит. Но в нужный тип значения переводят методом astype(), как раз его и выбираем.

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>
Хорошо

### Обработка дубликатов

Проверка полных дубликатов

In [29]:
df.duplicated().sum() 


54

Приведение данных к одному регистру

In [30]:
df['education'] = df['education'].str.lower() 
print(df['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Количество дубликатов после приведения данных к одному виду

In [31]:
df.duplicated().sum() 

71

Удаление дубликатов

In [32]:
df = df.drop_duplicates().reset_index(drop=True) 


In [33]:
df.duplicated().sum()

0

### Вывод

Первоначально количество дубликатов было равно 54, но некоторые были не замечены или различиях в регистре данных столбца education, после приведения к одному виду, количество увеличилось до 71, затем дубликаты были удалены методом drop_duplicates().reset_index.

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>
Все дубликаты найдены и обработаны верно,отлично


### Лемматизация

Лемматизируем столбец purpose 

In [34]:
from pymystem3 import Mystem
m = Mystem()
def lemmatizator(i):
    lemmas = m.lemmatize(i)
    return lemmas
df['purpose_lemmas'] = df['purpose'].apply(lemmatizator)

In [35]:
df['purpose_lemmas'].value_counts()

[автомобиль, \n]                                          961
[свадьба, \n]                                             785
[на,  , проведение,  , свадьба, \n]                       759
[сыграть,  , свадьба, \n]                                 755
[операция,  , с,  , недвижимость, \n]                     669
[покупка,  , коммерческий,  , недвижимость, \n]           655
[покупка,  , жилье,  , для,  , сдача, \n]                 647
[операция,  , с,  , коммерческий,  , недвижимость, \n]    643
[операция,  , с,  , жилье, \n]                            641
[покупка,  , жилье,  , для,  , семья, \n]                 636
[жилье, \n]                                               635
[покупка,  , жилье, \n]                                   634
[недвижимость, \n]                                        627
[строительство,  , собственный,  , недвижимость, \n]      626
[операция,  , со,  , свой,  , недвижимость, \n]           623
[строительство,  , недвижимость, \n]                      619
[покупка

### Вывод

Для того, чтобы в дальнейшем выделить категории целей кредита, для начала нам нужно было лемматизировать исходные цели первоначальной таблицы, после лемматизиции мы сохранили леммы в отдельный столбец

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>
Лемматизация проведена верно


### Категоризация данных

Категоризируем цели кредита:

In [36]:
def type_lemma(row): 
    
    purpose = row['purpose_lemmas']
    
    
    if 'автомобиль' in purpose :
        
        return 'автомобиль'
    
    if 'образование' in purpose :
        
        return 'образование'
    
    if 'свадьба' in purpose :
        
        return 'свадьба'
                
    else:
    
        return 'недвижимость'
    

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>
Верно всё

Применяем функцию и проверяем уникальные значения столбца

In [37]:
df['purpose_lemmas'] = df.apply(type_lemma, axis = 1) 
df['purpose_lemmas'].value_counts() 

недвижимость    10703
автомобиль       4258
образование      3970
свадьба          2299
Name: purpose_lemmas, dtype: int64

In [38]:
df_purpose = df

In [39]:
df_p2 = df_purpose.pivot_table(index='purpose_lemmas', values='debt')

In [40]:
df_p2['Процент возврата'] = 100 - df_p2['debt']*100
print(df_p2)

                    debt  Процент возврата
purpose_lemmas                            
автомобиль      0.093236         90.676374
недвижимость    0.072596         92.740353
образование     0.092947         90.705290
свадьба         0.078730         92.127012


Категоризация по наличию детей:

In [41]:
df_children = df

In [42]:
def children(row):
    children = row['children']
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей '

In [43]:
df_children['children'] = df_children.apply(children, axis = 1)

In [44]:
df['children'].value_counts()

нет детей     14021
есть дети      7209
Name: children, dtype: int64

In [45]:
df_c2 = df_children.pivot_table(index='children', values='debt')
print(df_c2)

                debt
children            
есть дети   0.092385
нет детей   0.075458


Категоризация по семейному статусу:

In [46]:
df_family = df

In [47]:
df_f2 = df_family.pivot_table(index='family_status', values='debt')

In [48]:
df_f2['Процент возврата'] = 100 - df_f2['debt']*100
print(df_f2)

                           debt  Процент возврата
family_status                                    
Не женат / не замужем  0.097842         90.215827
в разводе              0.071247         92.875318
вдовец / вдова         0.065539         93.446089
гражданский брак       0.093142         90.685798
женат / замужем        0.075575         92.442479


Категоризация по доходу:

Найдем медиальное и среднее значение дохода для всей таблицы:

In [49]:
print(df['total_income'].median())


142594.0


In [50]:
print(df['total_income'].mean())

165386.31832312766


Заметим, что медиальное значение дохода это в районе 140000, значит можно судить, что основная группа людей из нашей таблицы имеет доход близкий к 140000, но так как среднее значение больше и составляет 165000 значит, что есть группа лиц с доходом значительно превышающий медиальное значение. Возьмем небольшое отклонение в 50000 и найдем число людей с доходом от 100 до 200 тысяч:

In [51]:
len(df.query(' 100000 < total_income < 200000'))

11797

Таких людей больше 50% от общего числа. Проверим на наличие людей с высоким доходом от 200 тысяч и доходом меньше 100 тысяч

In [53]:
print(len(df.query('total_income > 200000')))
print(len(df.query('total_income < 100000')))

5012
4421


Получили, что оставшиеся 50% разделились в равных частях на тех, кто получает больше и на тех, кто меньше. Категоризируем по 3 этим группам:

In [54]:
df_total_income = df

In [55]:
def total_income(row):
    total_income = row['total_income']
    if total_income > 200000:
        return 'больше 200 тысяч'
    if   100000 < total_income < 200000:
        return 'от 150 до 200 тысяч'
    else:
        return 'меньше 100 тысяч '

In [56]:
df_total_income['total_income'] = df_total_income.apply(total_income, axis = 1)

In [57]:
df_total_income['total_income'].value_counts()

от 150 до 200 тысяч    11797
больше 200 тысяч        5012
меньше 100 тысяч        4421
Name: total_income, dtype: int64

In [58]:
df_t2 = df_total_income.pivot_table(index='total_income', values='debt')

In [59]:
df_t2['Процент возврата'] = 100 - df_t2['debt']*100
print(df_t2)

                         debt  Процент возврата
total_income                                   
больше 200 тысяч     0.070830         92.916999
меньше 100 тысяч     0.079846         92.015381
от 150 до 200 тысяч  0.086124         91.387641


### Вывод

По итогу мы категоризировали данные по наличию детей, выделили группы людей у которых есть дети и у которых их нет, по целям кредита, выделив 5 категорий( недвижимость, жилье, свадьба, автомобиль, образование), по семейному положению и по доходу

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошо

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [60]:
df_c2 = df_children.pivot_table(index='children', values='debt')
df_c2['Процент возврата'] = 100 - df_c2['debt']*100
print(df_c2)

                debt  Процент возврата
children                              
есть дети   0.092385         90.761548
нет детей   0.075458         92.454176


### Вывод

Люди у которых имеются дети отдают кредит реже это можно видеть по 2 столбцу 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [61]:
df_f2['Процент возврата'] = 100 - df_f2['debt']*100
print(df_f2)

                           debt  Процент возврата
family_status                                    
Не женат / не замужем  0.097842         90.215827
в разводе              0.071247         92.875318
вдовец / вдова         0.065539         93.446089
гражданский брак       0.093142         90.685798
женат / замужем        0.075575         92.442479


### Вывод

Люди которые состоят или когда состояли в официально офрмленных отношениях чаще отдают кредиты

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [62]:
df_t2['Процент возврата'] = 100 - df_t2['debt']*100
print(df_t2)

                         debt  Процент возврата
total_income                                   
больше 200 тысяч     0.070830         92.916999
меньше 100 тысяч     0.079846         92.015381
от 150 до 200 тысяч  0.086124         91.387641


### Вывод

Как таковой зависимости между возвратом кредита и уровнем дохода не замечено. Небольшая разница в том, что люди с большими доходами и группа с доходом меньше 100 тысяч отдают кредиты чаще, нежели медиальная группа с доходами от 100 до 200 тысяч. Но тут следует учитывать объем выборки, чем больше данных, тех больше точность.

- Как разные цели кредита влияют на его возврат в срок?

In [64]:
df_p2['Процент возврата'] = 100 - df_p2['debt']*100
print(df_p2)

                    debt  Процент возврата
purpose_lemmas                            
автомобиль      0.093236         90.676374
недвижимость    0.072596         92.740353
образование     0.092947         90.705290
свадьба         0.078730         92.127012


### Вывод

Покупка жилья, недвижимости или кредит на свадьбу дает лучшие показатели по возврату нежели образование и покупка автомобиля

<div style="border:solid  green  2px; padding: 20px">

<h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

Всё верно и точно сделано, как выводы, так и код написаны правильно




### Шаг 4. Общий вывод

Основной нашей задачей было узнать влияние различных факторов на платежеспособность клиентов банка. Мы проанализировали следующие факторы: наличие детей , семейное положние, цели кредита и уровень дохода. Если брать критерий наличие детей, казалось, что наличие ребенка должно как-то дисциплинировать и накладывать некую ответственность на родителей, но как показывает статистика, клиенты с детьми реже отдают кредит, возможно сказывается то, что дети это всегда какие-то непредвиденные расходы и в условиях ограниченного бюджета людям приходится просрочивать задолженности. Если судить по критерию семейного положения, те кто состоят или состояли в браке платят чаще, нежели свободные от отношений. Возможно это связано с некой дополнительной ответственностью не только за себя но и того человека с которым живешь. Если говорить про зависимость от дохода ,то существенного влияния на выплаты это не оказывает, вероятно единственное различие это будет размер кредита, который берут люди с разными уровнями дохода. И последний проанализированный нами факор это цели кредита, чем более глобальная покупка, тем более она обдуманная и тем больше вероятность своевременного погашения задолжности, если говорить про свадьбу, зачастую на нее берут кредиты с мыслью о том, что кто-то что-то подарит и смогут все погасить во время, в большинстве случаев так и происходит, поэтому процент возврата велик. 

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>
Отличный и цельный вывод, здорово!)


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.


- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  green  2px; padding: 20px">

<h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции, названия переменных передают смысл операций. В качестве совета предлагаю глубже изучить и начать применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем.

#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)
